# Implementing CellRanger

## Relevance:

Cellranger has a number of advantages over other tools for single-cell RNA-seq analysis to analize 10x Genomics data.   
It can automatically detect cells, guides and barcodes, and it can also automatically detect the read format.  
The matrix ouput can be transformed in a mudata object, which can be used for further analysis.

## Practical description:
- [x] Create a cell ranger nextflow process.
- [ ] Capture the inputs for guides and scRNA-seq and pre-format to be analyzed by cell ranger. If needed, create a special subdirs to move the data.
- [ ] Create the feature_ref.csv using the guides.xls information
- [ ] Return the mtx data.


This is a example of cellranger being used to process guides and rna-seq data. 
The file library.csv and feature_ref.csv

```bash 
 cellranger count --id=gasperini_01
--libraries=library.csv
--transcriptome=/opt/refdata-gex-GRCh38-2020-A
--feature-ref=feature_ref.csv 
```
The files library.csv and feature_ref.csv should be created automaticaly


obs:  
- For this jamboree we will assume that we will receive a single sample that is separated with in several files. Each small files contains read1 and read2. If it is needed these files can be merged in a single R1 and a single R2 file before use it in the cell ranger process.
- https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/using/feature-bc-analysis 


# Downloading reference and Cellranger (~ 6 minutes)

In [2]:
#Downloading cell ranger ~3 minutes
!wget -O cellranger-7.1.0.tar.gz "https://cf.10xgenomics.com/releases/cell-exp/cellranger-7.1.0.tar.gz?Expires=1680506161&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZi4xMHhnZW5vbWljcy5jb20vcmVsZWFzZXMvY2VsbC1leHAvY2VsbHJhbmdlci03LjEuMC50YXIuZ3oiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2ODA1MDYxNjF9fX1dfQ__&Signature=Wl9R1wkyTquf24HwsdxIWlXm2d9FsXSNmuhs7XXYQTl1qkwkobt2fBN3jMr~LPQQBrVIWPAgCvqmLVdrq-moQgoUa9p5Kc77btjk-EopvYTa3zvNTPnOJ9sc2IGQejhFgDYtLSwr210hfqdkVc7WIwM4yCe~oQSyWSad~vibjS-y8Kvfn0~jlLxIDMkFXL9Nhq9cgLV9tkI3HcamWfTkTuSRYgr~U6AlBcWAAiJa7OVJTJLNWJtIDqvaMEdTT1yM7DLfoooRmBeMi2nS~k5a-pBhOAyxWN2i5gwyzKIwuWDQvon0WwT-8XpHLgM1OO7aXWYtBA88f06eZNJF4rJN3g__&Key-Pair-Id=APKAI7S6A5RYOXBWRPDA"
!tar -xzvf cellranger-7.1.0.tar.gz


--2023-04-02 19:16:31--  https://cf.10xgenomics.com/releases/cell-exp/cellranger-7.1.0.tar.gz?Expires=1680506161&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZi4xMHhnZW5vbWljcy5jb20vcmVsZWFzZXMvY2VsbC1leHAvY2VsbHJhbmdlci03LjEuMC50YXIuZ3oiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2ODA1MDYxNjF9fX1dfQ__&Signature=Wl9R1wkyTquf24HwsdxIWlXm2d9FsXSNmuhs7XXYQTl1qkwkobt2fBN3jMr~LPQQBrVIWPAgCvqmLVdrq-moQgoUa9p5Kc77btjk-EopvYTa3zvNTPnOJ9sc2IGQejhFgDYtLSwr210hfqdkVc7WIwM4yCe~oQSyWSad~vibjS-y8Kvfn0~jlLxIDMkFXL9Nhq9cgLV9tkI3HcamWfTkTuSRYgr~U6AlBcWAAiJa7OVJTJLNWJtIDqvaMEdTT1yM7DLfoooRmBeMi2nS~k5a-pBhOAyxWN2i5gwyzKIwuWDQvon0WwT-8XpHLgM1OO7aXWYtBA88f06eZNJF4rJN3g__&Key-Pair-Id=APKAI7S6A5RYOXBWRPDA
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 669216050 (638M) [application/x-

In [3]:
#Downloading reference ~ 3 minutes
!curl -O https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz
!tar -xzvf refdata-gex-GRCh38-2020-A.tar.gz
!rm refdata-gex-GRCh38-2020-A.tar.gz
!rm cellranger-7.1.0.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.6G  100 10.6G    0     0   147M      0  0:01:14  0:01:14 --:--:--  159M


In [10]:
!pwd

/home/jovyan


In [48]:
!ls refdata-gex-GRCh38-2020-A

737K-august-2016.txt		 my_mu_mod.py
737K-august-2016.txt.1		 pipeline_perturbseq_like
assign_guide.py			 refdata-gex-GRCh38-2020-A
cellranger-7.1.0		 refdata-gex-GRCh38-2020-A.tar.gz
cellranger-7.1.0.tar.gz		 results_track_visualization.nf
cellranger.nf			 run_jamboree_v1.ipynb
gasperini_perturb.config	 task2_hub.ipynb
gasperini_test_01		 Task4.ipynb
gasperini_test_01.html		 task_cellranger.ipynb
generating_cell_range_inputs.py  Task_guide_assign_V4.ipynb
guide_assign.nf			 test_cellranger
guide_assing_test		 testing_mu_mod_workflow_test
hon_guide_jamboree.yaml		 test_transfer
hon_guide.yaml			 track_generation.py
lost+found			 track_test_01
mudata_results.h5mu		 Untitled.ipynb
mu_mod_workflow_test.nf


In [ ]:
!git clone https://github.com/LucasSilvaFerreira/pipeline_perturbseq_like.git 

# Template 

In [45]:
%%writefile cellranger.nf


params.CELLRANGER_REF = '/home/jovyan/refdata-gex-GRCh38-2020-A'
params.GUIDE_FEATURES = '/home/jovyan/pipeline_perturbseq_like/df_from_gasperini_tss.xlsx'
params.CHEMISTRY = '10xv2'
params.WHITELIST= '/home/jovyan/737K-august-2016.txt'
params.FASTQ_FILES_TRANSCRIPTS = ['/mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L002_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L002_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L003_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L003_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L004_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L004_R2_001.fastq.gz'] 
params.FASTQ_NAMES_TRANSCRIPTS = ['S1_L1'] 
params.FASTQ_FILES_GUIDES = ['/mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L004_R2_001.fastq.gz'] 
params.FASTQ_NAMES_GUIDES = ['S1_L1'] 





process cellranger {
        cpus 5
        debug true
        input:
        tuple val(rna_sample_name)
        tuple val(rna_string_fastqz)
        tuple val(guide_sample_name)
        tuple val(guide_string_fastqz)
        tuple val(guide_features)
        tuple val (cellranger_ref)
        tuple val(chemistry)
        tuple val(threads)
        tuple val(whitelist)
        output:
        path ('outs/filtered_feature_bc_matrix/matrix.mtx.gz'),  emit: mtx_file

        script:
    
        """
        
        chmod 700 /home/jovyan/generating_cell_range_inputs.py
        /home/jovyan/generating_cell_range_inputs.py --rna_sample_name $rna_sample_name --rna_string_fastqz "$rna_string_fastqz"  --guide_sample_name $guide_sample_name   --guide_string_fastqz  "$guide_string_fastqz"  --guide_features $guide_features  --chemistry  $chemistry  --whitelist $whitelist
        #The script generating_cell_range_inputs.py should create the library.csv and feature_ref.csv 
        cellranger count --id=gasperini_01 --libraries=library.csv --transcriptome=$cellranger_ref --feature-ref=feature_ref.csv 
        
        
        
        #Artificially creating the output. Remove it when cellranger starts running.
        
        mkdir  outs
        mkdir  outs/filtered_feature_bc_matrix/
        mkdir 'outs/filtered_feature_bc_matrix/matrix.mtx.gz'
        
        """
} 
        


workflow {
    
      cellranger_out = cellranger (
                 Channel.from(params.FASTQ_NAMES_TRANSCRIPTS),
                 Channel.from(params.FASTQ_FILES_TRANSCRIPTS),
                 Channel.from(params.FASTQ_NAMES_GUIDES),
                 Channel.from(params.FASTQ_FILES_GUIDES),
                 Channel.from(params.GUIDE_FEATURES),
                 Channel.of(params.CHEMISTRY).collect(),
                 Channel.of(params.THREADS).collect(),
                 Channel.of(params.WHITELIST).collect(),
                )



}


Overwriting cellranger.nf


In [46]:
%%writefile  generating_cell_range_inputs.py
#!/usr/bin/env python
import argparse

parser = argparse.ArgumentParser(description='Description of your program')
parser.add_argument('--rna_sample_name', help='Description of the RNA sample name variable')
parser.add_argument('--rna_string_fastqz', help='Description of the RNA string fastqz variable')
parser.add_argument('--guide_sample_name', help='Description of the guide sample name variable')
parser.add_argument('--guide_string_fastqz',  help='Description of the string fastqz variable')
parser.add_argument('--guide_features',  help='Description of the guide features variable')
parser.add_argument('--chemistry',  help='Description of the chemistry variable')
parser.add_argument('--threads', type=int, help='Description of the threads variable')
parser.add_argument('--whitelist', help='Description of the whitelist variable')

args = parser.parse_args()

print(args.rna_sample_name)
print(args.rna_string_fastqz)
print(args.guide_sample_name)
print(args.guide_string_fastqz)
print(args.guide_features)
print(args.chemistry)
print(args.threads)
print(args.whitelist)

#ex:




#Use these variables to create the inputs for  cellranger.


Overwriting generating_cell_range_inputs.py


In [47]:
!nextflow run cellranger.nf -w test_cellranger

N E X T F L O W  ~  version 22.10.6
Launching `cellranger.nf` [scruffy_torvalds] DSL2 - revision: fd8748f3e7
WARN: Access to undefined parameter `THREADS` -- Initialise it to a default value eg. `params.THREADS = some_value`
WARN: Input `set` must define at least two component -- Check process `cellranger`
[-        ] process > cellranger -

executor >  local (1)
[43/37b3a5] process > cellranger (1) [100%] 1 of 1 ✔
S1_L1
/mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  /mnt/shared/data/bam_pilot_scrna_1/K1000_CRISPRi_cells_

# SANDBOX

In [4]:
#executing cell ranger
!./cellranger-7.1.0/bin/cellranger  

refdata-gex-GRCh38-2020-A/
refdata-gex-GRCh38-2020-A/pickle/
refdata-gex-GRCh38-2020-A/pickle/genes.pickle
refdata-gex-GRCh38-2020-A/fasta/
refdata-gex-GRCh38-2020-A/fasta/genome.fa.fai
refdata-gex-GRCh38-2020-A/fasta/genome.fa
refdata-gex-GRCh38-2020-A/star/
refdata-gex-GRCh38-2020-A/star/transcriptInfo.tab
refdata-gex-GRCh38-2020-A/star/chrNameLength.txt
refdata-gex-GRCh38-2020-A/star/SAindex
refdata-gex-GRCh38-2020-A/star/geneInfo.tab
refdata-gex-GRCh38-2020-A/star/SA
refdata-gex-GRCh38-2020-A/star/exonInfo.tab
refdata-gex-GRCh38-2020-A/star/chrStart.txt
refdata-gex-GRCh38-2020-A/star/chrName.txt
refdata-gex-GRCh38-2020-A/star/sjdbList.fromGTF.out.tab
refdata-gex-GRCh38-2020-A/star/chrLength.txt
refdata-gex-GRCh38-2020-A/star/sjdbInfo.txt
refdata-gex-GRCh38-2020-A/star/genomeParameters.txt
refdata-gex-GRCh38-2020-A/star/exonGeTrInfo.tab
refdata-gex-GRCh38-2020-A/star/Genome
refdata-gex-GRCh38-2020-A/star/sjdbList.out.tab
refdata-gex-GRCh38-2020-A/genes/
refdata-gex-GRCh38-2020-A/gen

cellranger cellranger-7.1.0
Process 10x Genomics Gene Expression, Feature Barcode, and Immune Profiling data

USAGE:
    cellranger <SUBCOMMAND>

OPTIONS:
    -h, --help       Print help information
    -V, --version    Print version information

SUBCOMMANDS:
    count               Count gene expression (targeted or
                            whole-transcriptome) and/or feature barcode reads
                            from a single sample and GEM well
    multi               Analyze multiplexed data or combined gene
                            expression/immune profiling/feature barcode data
    multi-template      Output a multi config CSV template
    vdj                 Assembles single-cell VDJ receptor sequences from
                            10x Immune Profiling libraries
    aggr                Aggregate data from multiple Cell Ranger runs
    reanalyze           Re-run secondary analysis (dimensionality reduction,
                            clustering, etc)
    targeted-c